<a href="https://colab.research.google.com/github/victwise/fastai_numerical_linear_algebra/blob/master/Cap%C3%ADtulo6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy, numpy as np
from scipy import linalg

In [0]:
np.set_printoptions(precision=6)

In [0]:
data = datasets.load_diabetes()

In [0]:
data = datasets.load_diabetes()

In [0]:
trn,test,y_trn,y_test = train_test_split(data.data, data.target, test_size=0.2)

In [9]:
trn.shape, test.shape

((353, 10), (89, 10))

In [0]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred))